In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import time

In [2]:
PEOPLE =['Alice', 'Bob', 'Carol', 'Dave', 'Eve']
GIFTS= ['Book', 'Toy', 'Chocolate', 'Wine', 'Flowers']
GIFTCOSTS=[ 10, 20, 5, 15, 7]
HAPPINESS={
'Book': [3, 2, 5, 1, 4],
'Toy': [5, 2, 4, 3, 1],
'Chocolate': [1, 3, 4, 5, 2],
'Wine': [2, 5, 3, 4, 1],
'Flowers': [4, 3, 1, 2, 5]}
BUDGET= 50

$$ \max_{X} \sum_{p,g} X_{p,g}H_{p,g} $$ 

$$ \sum_{p,g} X_{p,g}Cost_g \leq Budget$$ 
$$ \forall p \ \sum_{g} X_{p,g} = 1$$ 


In [3]:
model = AbstractModel()
model.p = Set(initialize=PEOPLE)
model.g = Set(initialize=GIFTS)
model.x = Var(model.p,model.g, within=Binary)
def H_rule(model, p,g):
    return HAPPINESS[g][1]
model.happiness = Param(model.p,model.g, rule=H_rule)

def C1_rule(model):
    return sum(model.x[p,g]*GIFTCOSTS[GIFTS.index(g)] for p in model.p for g in model.g) <= BUDGET
model.C1 = Constraint(rule=C1_rule)

def C2_rule(model,p):
    return sum(model.x[p,g] for g in model.g) == 1
model.C2 = Constraint(model.p,rule=C2_rule)

def obj_rule(model):
    return sum(model.x[p,g]*HAPPINESS[g][PEOPLE.index(p)] for p in model.p for g in model.g) 
model.OF = Objective(rule=obj_rule,sense=maximize)

In [8]:
opt = SolverFactory('glpk')
instance = model.create_instance()

start = time.time()
results = opt.solve(instance) # solves and updates instance
print('Time is ', round(time.time() - start, 4) ) 

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)
print('Total Happiness is ', value(instance.OF))

Time is  0.03
feasible
Total Happiness is  24.0


In [7]:
for p in instance.p:
    for g in instance.g:
        if value(instance.x[p,g])>0:
            print(p,' ----->  ', g)

Alice  ----->   Flowers
Bob  ----->   Wine
Carol  ----->   Book
Dave  ----->   Chocolate
Eve  ----->   Flowers
